# KGCN 논문 리뷰 & 코드 작성
**KGCN: Simplifying and Powering Graph Convolution Network for Recommendation**  
*Hongwei Wang et al. (2019)*  
🔗 [논문 링크](https://arxiv.org/abs/1904.12575)

## 3.1 Problem Formulation

*목표 : 사용자 u가 아이템 v에 관심 있을지를 예측하는 함수*
$$\hat{y}_{uv} = F(u,v | Θ, Y, G)$$
- Y : 사용자-아이템 상호작용 행렬 (예: 클릭, 평가)
- G : 지식 그래프 (KG), 삼중항(triple : head, relation, tail)의 집합

## 3.2 KGCN Layer (모델 구성 요소)

*1. 관계 중요도 계산 (사용자 u, 관계 r)*
$$\pi_{ur} = g(u,r)$$

*2. 이웃 노드 집계 (attention 가중치 포함)*
$$v_u^{N(v)} = \sigma_{e\in{N(v)}}\tilde{\pi}_{ur}⋅e$$
$$\tilde{\pi}_{ur}=\frac{exp(\pi_{ur})}{\sum\nolimits_{e'}exp(\pi_{ur'})}$$

*3. Aggregation 방식 3가지*
- Sum : $ReLU(W(v+v_u^{N(v)})+b)$
- Concat : $ReLU(W[v;v_u^{N(v)}]+b)$
- Neighbor : $ReLU(Wv_u^{N(v)}+b)$

## 3.3 Learning Algorithm (학습 알고리즘)

*반복 구조*
KGCN은 여러 계층(hop)으로 구성되어 '0-hop → 1-hop → ...'와 같은 형식으로 이웃 정보를 반복적으로 전파 및 집계
$$\hat{y}_{uv}=f(u, v_u^{(h)})$$

*학습 손실 함수*
- Cross Entropy + Negative Sampling + L2 정규화 포함
$$L = \sum_{u}\begin{bmatrix}\sum_{v:y_{uv}=1}J(y_{uv}, \hat{y}_{uv})-\sum_{i=1}^{T_u}\mathbb{E}_{vi~P(v)}J(0,\hat{y}_(uvi))\end{bmatrix}+\lambda||F||_2^2$$

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import random

In [ ]:
device = (torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu"))
print("Using device:", device)

## Book - Crossing 모델 설정 (Hyperparameter)

|항목|설정|
|:---:|:---:|
임베딩 차원 | 64
학습률 | 0.0002
Optimizer | Adam
정규화 계수 (L2) | 2e-5
Negative Sampling | 1:1 비율로 샘플링
배치 사이즈 | 256
학습 Epoch | 최대 1000 (일반적으로 200~400)
레이어 수 (Receptive Field Depth, H) | 1
이웃 샘플링 수 (K) | 8
초기화 방식 | Xavier Uniform (명시는 없지만 일반적인 초기화 방식으로 추정됨)

In [4]:
config = {
    'device': 'mps',
    'dataset': 'book',
    'embedding_dim': 64,
    'n_layers': 1,
    'lr': 0.0002,
    'batch_size': 256,
    'l2': 2e-5,
    'n_epoch': 200,
    'n_neighbor': 8,
    'H': 1,
    'aggregator': 'sum'
}

In [ ]:
def load_kg(file_path):
    kg = dict()
    with open(file_path, 'r') as f:
        for line in f:
            h, r, t = map(int, line.strip().split())
            if h not in kg:
                kg[h] = []
            kg[h].append((t, r))
            if t not in kg:
                kg[t] = []
            kg[t].append((h, r))
    return kg

In [ ]:
class KGCN(nn.Module):
    def __init__(self, n_users, n_entities, n_relations, kg, config):
        super(KGCN, self).__init__()
        self.embedding_dim = config['embedding_dim']
        self.n_layers = config['n_layers']
        self.n_neighbors = config['n_neighbor']
        self.aggregator_type = config['aggregator']
        self.device = config['device']
        self.kg = kg

        self.user_emb = nn.Embedding(n_users, self.embedding_dim)
        self.entity_emb = nn.Embedding(n_entities, self.embedding_dim)
        self.relation_emb = nn.Embedding(n_relations, self.embedding_dim)

        self.W = nn.Linear(self.embedding_dim, self.embedding_dim)
        self.activation = nn.ReLU()

        self.to(self.device)

    def aggregate(self, entity_vectors, neighbor_vectors, neighbor_relations, user_vector):
        user_vector = user_vector.unsqueeze(1)
        scores = torch.sum(user_vector * neighbor_relations, dim=-1, keepdim=True)
        att_weights = torch.softmax(scores, dim=1)
        weighted_neighbors = torch.sum(att_weights * neighbor_vectors, dim=1)

        if self.aggregator_type == 'sum':
            out = entity_vectors + weighted_neighbors
        elif self.aggregator_type == 'concat':
            out = torch.cat([entity_vectors, weighted_neighbors], dim=-1)
            out = self.W(out)
        elif self.aggregator_type == 'neighbor':
            out = weighted_neighbors
        else:
            raise ValueError("Unknown aggregator type")

        return self.activation(out)

    def forward(self, user_ids, item_ids):
        user_vector = self.user_emb(user_ids)
        entity_vector = self.entity_emb(item_ids)
        neighbor_vectors = self.get_neighbors(item_ids, hop=1)

        for _ in range(self.n_layers):
            entity_vector = self.aggregate(entity_vector, *neighbor_vectors, user_vector)
            neighbor_vectors = self.get_neighbors(item_ids, hop=1)

        scores = torch.sum(entity_vector * user_vector, dim=1)
        return torch.sigmoid(scores)

    def get_neighbors(self, entity_ids, hop=1):
        batch_neighbors = []
        batch_relations = []

        for eid in entity_ids.cpu().numpy():
            neighbors = self.kg.get(eid, [])
            if len(neighbors) == 0:
                neighbors = [(eid, 0)] * self.n_neighbors
            elif len(neighbors) < self.n_neighbors:
                neighbors = neighbors + random.choices(neighbors, k=self.n_neighbors - len(neighbors))
            else:
                neighbors = random.sample(neighbors, self.n_neighbors)

            e_ids = [e for e, r in neighbors]
            r_ids = [r for e, r in neighbors]
            batch_neighbors.append(e_ids)
            batch_relations.append(r_ids)

        neighbor_entities = torch.tensor(batch_neighbors, device=self.device)
        neighbor_relations = torch.tensor(batch_relations, device=self.device)

        neighbor_entity_emb = self.entity_emb(neighbor_entities)
        neighbor_relation_emb = self.relation_emb(neighbor_relations)

        return neighbor_entity_emb, neighbor_relation_emb